In [ ]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 169kB/s 
     |████████████████████████████████| 421.8MB 34kB/s 
     |████████████████████████████████| 983kB 20.0MB/s 
     |████████████████████████████████| 245kB 31.3MB/s 
     |████████████████████████████████| 778kB 27.3MB/s 
     |████████████████████████████████| 471kB 34.1MB/s 
     |████████████████████████████████| 3.9MB 35.8MB/s 
     |████████████████████████████████| 450kB 34.7MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 3.0MB 25.8MB/s 
     |████████████████████████████████| 1.1MB 30.8MB/s 
     |████████████████████████████████| 890kB 21.3MB/s 
  Created wheel for ktrain: filename=ktrain-0.20.0-cp36-none-any.whl size=25269082 sha256=78945e718efdd062b1f6d51b51a52144e06734cb17d7bb530e8f81a922c6af01
  Stored in directory: /root/.cache/pip/wheels/0d/90/b9/7c4d11547b37c382e59d98c3587c53364971a4547b913fa9b1
  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none

In [ ]:
import numpy as np 
import pandas as pd
import tensorflow as tf
import ktrain
from ktrain import text
import tensorflow as tf

In [ ]:
print(tf.__version__)

2.1.0


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving training.1600000.processed.noemoticon.csv to training.1600000.processed.noemoticon.csv


In [ ]:
#import csv into dataframe 

df_train = pd.read_csv('testdata.manual.2009.06.14.csv', dtype= str)
df = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding = "ISO-8859-1", header=None, dtype= str).iloc[:, [0, 4, 5]].sample(frac=1).reset_index(drop=True)



In [ ]:
df.head()

,0,4,5
0,0,Mystikk,Man it's two o clock 2 more hrs till I have to...
1,4,ayanyanks,IM SORRY Y'ALL. esp.@segundemuren and @BankyW....
2,0,TraceThurman,p.s. I miss my friends in Conway...
3,0,JustRelli,I just lost a follower
4,0,MsSchuld,@Da1n0nly seriously lol OMG he deleted you?? W...


In [ ]:
df.tail()

,0,4,5
1599995,0,krissenbee,Cute text... But thinking about boys is no fun...
1599996,0,naqiuddin,doing : alignment + bold title content dapat...
1599997,0,manny_miles,i didn't get to see a photo of lil d's party d...
1599998,0,SophieeOx,Doing English Homework
1599999,4,chryssa_m,"@COOTIEFREE wait, people go to bed before 4am?"


In [ ]:
for col in df.columns: 
    print(col) 

In [ ]:
df[5].head()

0    Man it's two o clock 2 more hrs till I have to...
1    IM SORRY Y'ALL. esp.@segundemuren and @BankyW....
2                 p.s. I miss my friends in Conway... 
3                              I just lost a follower 
4    @Da1n0nly seriously lol OMG he deleted you?? W...
Name: 5, dtype: object

In [ ]:
#from keras.preprocessing import sequence
import re

def preprocess_tweet(tweet):
    #Preprocess the text in a single tweet
    #arguments: tweet = a single tweet in form of string 
    #convert the tweet to lower case
    tweet.lower()
    #convert all urls to sting "URL"
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #convert all @username to "AT_USER"
    tweet = re.sub('@[^\s]+','AT_USER', tweet)
    #correct all multiple white spaces to a single white space
    tweet = re.sub('[\s]+', ' ', tweet)
    #convert "#topic" to just "topic"
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    tweet = re.sub(r'\W*\b\w{1,3}\b', '', tweet)
    return tweet



df[5] = df[5].apply(preprocess_tweet)

In [ ]:
df.head()

,0,4,5
0,0,Mystikk,clock more till have work
1,4,ayanyanks,SORRY.AT_USER AT_USER blame lettin much!!! so...
2,0,TraceThurman,miss friends Conway...
3,0,JustRelli,just lost follower
4,0,MsSchuld,AT_USER seriously deleted hurt even good ones


In [ ]:
#rename the column names
df.columns = ['target','name','text']

In [ ]:
#remove name column
df_new = df.drop(columns=['name'])

In [ ]:
# shaf
df_new = df.sample(frac=1)

In [ ]:
df_new.head()

,target,name,text
784206,4,gautamghosh,AT_USER wonder
1322563,0,JulienFrisch,Have cancel travel plans Daugavpils-related re...
273288,4,NickMcLaren,AT_USER AT_USER shall rest!
257386,4,ramezm,AT_USER welcome
458487,4,imaladysailor,warm hugs from nice boys


In [ ]:
df_train = df_new[:50000]
df_train = df_train.drop(columns=['name'])

train = df_train[:25000]
test = df_train[25000:]

In [ ]:
train.shape

(25000, 2)

In [ ]:
test.shape

(25000, 2)

In [ ]:
df_train.shape

In [ ]:
test['text']

In [ ]:
(X_train, y_train), (X_test, y_test), preproc = text.texts_from_df(
                   train_df = train,
                   text_column ='text',
                   label_columns = ['target'],
                   val_df = test, 
                   maxlen = 400,
                   preprocess_mode = 'bert'  
                   )

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [ ]:
X_train[0].shape

(25000, 400)

In [ ]:
model = text.text_classifier(name='bert',
                             train_data = (X_train, y_train),
                             preproc = preproc
                             
                             )

Is Multi-Label? False
maxlen is 400
done.


In [ ]:
learner = ktrain.get_learner(model = model, 
                              train_data = (X_train, y_train),
                             val_data = (X_test, y_test),
                             batch_size = 6
                             
                             )

In [ ]:
learner.lr_find()
learner.lr_plot()

simulating training for different learning rates... this may take a few moments...
Train on 25000 samples
Epoch 1/1024
  222/25000 [..............................] - ETA: 70:22:06 - loss: 0.7529 - accuracy: 0.50 - ETA: 41:10:51 - loss: 0.6372 - accuracy: 0.66 - ETA: 31:19:25 - loss: 0.6456 - accuracy: 0.66 - ETA: 26:19:11 - loss: 0.6513 - accuracy: 0.66 - ETA: 23:20:39 - loss: 0.6626 - accuracy: 0.63 - ETA: 21:27:36 - loss: 0.6892 - accuracy: 0.61 - ETA: 20:09:10 - loss: 0.6985 - accuracy: 0.61 - ETA: 19:08:47 - loss: 0.7168 - accuracy: 0.60 - ETA: 18:19:51 - loss: 0.7348 - accuracy: 0.59 - ETA: 17:42:46 - loss: 0.7346 - accuracy: 0.60 - ETA: 17:10:44 - loss: 0.7579 - accuracy: 0.57 - ETA: 16:44:21 - loss: 0.7467 - accuracy: 0.58 - ETA: 16:23:49 - loss: 0.7693 - accuracy: 0.56 - ETA: 16:05:59 - loss: 0.7897 - accuracy: 0.54 - ETA: 15:47:31 - loss: 0.7904 - accuracy: 0.54 - ETA: 15:31:16 - loss: 0.7731 - accuracy: 0.56 - ETA: 15:17:15 - loss: 0.7576 - accuracy: 0.57 - ETA: 15:04:43 - lo